In [1]:
from bs4 import BeautifulSoup
import requests, time
import pymongo
from splinter import Browser
import pandas

In [2]:
mars_news_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
jpl_featured_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

In [3]:
response = requests.get(mars_news_url)
soup = BeautifulSoup(response.text, 'lxml')

In [4]:
title = soup.find('div', class_='content_title').text

In [5]:
article_teaser = soup.find('div', class_='rollover_description_inner').text

In [6]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)

UnboundLocalError: local variable 'e' referenced before assignment

In [7]:
browser.visit(jpl_featured_image_url)

In [8]:
image_soup = BeautifulSoup(browser.html, "html.parser")
image_url = "https://www.jpl.nasa.gov" + image_soup.find('a', id="full_image").get("data-fancybox-href")

In [9]:
browser.visit("https://twitter.com/marswxreport?lang=en")
time.sleep(5)

In [10]:
weather_soup = BeautifulSoup(browser.html, "html.parser")
weather_tweet = weather_soup.find_all("span", class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0")

In [11]:
mars_weather = weather_tweet[27].text

In [12]:
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)

In [13]:
mars_facts_html = BeautifulSoup(browser.html, "html.parser")
mars_table = mars_facts_html.find("table", class_="tablepress tablepress-id-p-mars")

In [14]:
mars_df = pandas.read_html(str(mars_table))
mars_df_table = mars_df[0]

mars_df_table.rename(columns = {0: "description",
                                1: "value"},
                    inplace = True)
mars_table_html = mars_df_table.to_html()

In [15]:
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)
time.sleep(4)

In [16]:
hemispheres = BeautifulSoup(browser.html, "html.parser")
#print(hemispheres.find_all("a", class_= "itemLink product-item"))
hemisphere_list = hemispheres.find_all("h3")

In [17]:
hemisphere_image_urls = []
for x in range(0, len(hemisphere_list)):
    browser.click_link_by_partial_text(hemisphere_list[x].text)
    hemisphere_soup = BeautifulSoup(browser.html,"html.parser")
    hemisphere_dict = {"title":hemisphere_soup.find('h2').text.rsplit(' ', 1)[0],
                       "img_url":hemisphere_soup.find_all('li')[1].find('a').get("href")   
    }
    hemisphere_image_urls.append(hemisphere_dict)
    time.sleep(3)
    browser.back()

/opt/anaconda3/envs/PyhtonData/lib/python3.6/site-packages/splinter/driver/webdriver/__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [18]:
browser.quit()

In [19]:
mars_weather

'InSight sol 525 (2020-05-18) low -93.6ºC (-136.5ºF) high -0.4ºC (31.3ºF)\nwinds from the WNW at 4.8 m/s (10.8 mph) gusting to 18.5 m/s (41.4 mph)\npressure at 7.10 hPa'